In [ ]:
import os
import shutil

# Get user profile path
user_profile = os.environ["USERPROFILE"]

# Paths to Hugging Face cached models
cached_models = [
    os.path.join(user_profile, r".cache\huggingface\hub\models--bert-base-multilingual-cased"),
    os.path.join(user_profile, r".cache\huggingface\hub\models--xlm-roberta-base")
]

# Remove cached models if they exist
for path in cached_models:
    if os.path.exists(path):
        shutil.rmtree(path)
        print(f"Removed cache: {path}")
    else:
        print(f"No cache found at: {path}")


In [1]:
# Set device
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"

In [2]:
# ==============================
# Imports
# ==============================
import torch
import pandas as pd
import random
import numpy as np
from collections import Counter
from transformers import XLMRobertaTokenizer, XLMRobertaForMaskedLM
from openprompt.prompts import ManualTemplate, ManualVerbalizer
from openprompt.data_utils import InputExample
from openprompt.plms import load_plm
from openprompt import PromptForClassification, PromptDataLoader
from torch.optim import AdamW
from sklearn.metrics import classification_report
from collections import defaultdict
from torch.utils.data import DataLoader, Sampler

# ========================================
# Check CUDA
# ========================================
device = "cuda" #if torch.cuda.is_available() else "cpu"
print("CUDA available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("GPU name:", torch.cuda.get_device_name(0))
    print("CUDA version:", torch.version.cuda)
    print("GPU count:", torch.cuda.device_count())

# ========================================
# Seeds for reproducibility
# ========================================
seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

C:\Users\stdFurqan\anaconda3\envs\py310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


CUDA available: True
GPU name: NVIDIA GeForce RTX 4080 SUPER
CUDA version: 12.1
GPU count: 1


In [3]:
### Classes ###
classes = ['entailment', 'neutral', 'contradiction']

### Label Map ###
label_map = {'entailment': 0, 'neutral': 1, 'contradiction': 2}



# # Step 1: Use load_plm with 'roberta' to get the correct WrapperClass
_, _, _, WrapperClass = load_plm("roberta", "roberta-base")  # Just to get the wrapper

# # # Step 2: Manually load XLM-RoBERTa model/tokenizer
model_name = "xlm-roberta-base"
tokenizer = XLMRobertaTokenizer.from_pretrained(model_name)
plm = XLMRobertaForMaskedLM.from_pretrained(model_name)

# ==============================
# Load Pretrained Language Model (mBERT)
# ==============================
# plm, tokenizer, model_config, WrapperClass = load_plm("bert", "bert-base-multilingual-cased")

C:\Users\stdFurqan\anaconda3\envs\py310\lib\site-packages\huggingface_hub\file_download.py:945: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [4]:
# ==============================
# Define Prompt Template (Manual)
# ==============================
template = ManualTemplate(
    text = '{"placeholder":"text_a"} اور {"placeholder":"text_b"} کا تعلق {"mask"} ہے۔',
    tokenizer=tokenizer,
)



# templates = [
#     ("P1", ManualTemplate(
#         text='{"placeholder":"text_a"} اور {"placeholder":"text_b"} کا تعلق {"mask"} ہے۔',
#         tokenizer=tokenizer,
#     )),

#     ("P2", ManualTemplate(
#         text='پہلا جملہ: {"placeholder":"text_a"} دوسرا جملہ: {"placeholder":"text_b"} ان دونوں کے درمیان تعلق {"mask"} ہے۔',
#         tokenizer=tokenizer,
#     )),

#     ("P3", ManualTemplate(
#         text='{"placeholder":"text_a"} کے مقابلے میں {"placeholder":"text_b"} کا مفہوم {"mask"} ہے۔',
#         tokenizer=tokenizer,
#     )),

#     ("P4", ManualTemplate(
#         text='اگر {"placeholder":"text_a"} درست ہے تو {"placeholder":"text_b"} {"mask"} ہے۔',
#         tokenizer=tokenizer,
#     )),

#     ("P5", ManualTemplate(
#         text='{"placeholder":"text_b"} کا بیان {"placeholder":"text_a"} کے لحاظ سے {"mask"} ہے۔',
#         tokenizer=tokenizer,
#     )),

#     ("P6", ManualTemplate(
#         text='{"placeholder":"text_a"} اور {"placeholder":"text_b"} کے درمیان رشتہ {"mask"} قرار پاتا ہے۔',
#         tokenizer=tokenizer,
#     )),

#     ("P7", ManualTemplate(
#         text='پہلے جملے {"placeholder":"text_a"} کی بنیاد پر دوسرا جملہ {"placeholder":"text_b"} {"mask"} ہے۔',
#         tokenizer=tokenizer,
#     )),

#     ("P8", ManualTemplate(
#         text='{"placeholder":"text_a"} کے تناظر میں {"placeholder":"text_b"} کا تعلق {"mask"} بنتا ہے۔',
#         tokenizer=tokenizer,
#     )),

#     ("P9", ManualTemplate(
#         text='{"placeholder":"text_a"} کے بعد {"placeholder":"text_b"} کا نتیجہ {"mask"} سمجھا جاتا ہے۔',
#         tokenizer=tokenizer,
#     )),

#     ("P10", ManualTemplate(
#         text='{"placeholder":"text_a"} اور {"placeholder":"text_b"} کے درمیان منطقی تعلق {"mask"} ہے۔',
#         tokenizer=tokenizer,
#     )),
# ]






verbalizer = ManualVerbalizer(
    classes=classes,
    label_words={
        "entailment": ["درست", "لازم"],
        "neutral": ["ممکن", "غیر واضح"],
        "contradiction": ["غلط", "متضاد"]
    },
    tokenizer=tokenizer,
)


In [5]:
# ==============================
# Create Prompt Model
# ==============================
prompt_model = PromptForClassification(
    template=template,
    plm=plm,
    verbalizer=verbalizer
)


In [6]:
# ==============================
# Load Evaluation Dataset
# ==============================
df = pd.read_csv(
    r"C:\Users\stdFurqan\Desktop\paft\XNLI\XNLI_Dev_Urdu - Sheet1 (1) - XNLI_Dev_Urdu - Sheet1 (1).tsv.csv"
)

# Make InputExamples
eval_dataset = [
    InputExample(
        guid=i,
        text_a=row['sentence1'],
        text_b=row['sentence2'],
        label=label_map[row['gold_label']]
    )
    for i, row in df.iterrows()
]



# ==============================
# PromptDataLoader
# ==============================
eval_loader = PromptDataLoader(
    dataset=eval_dataset,
    tokenizer=tokenizer,
    template=template,              # your ManualTemplate for XNLI
    tokenizer_wrapper_class=WrapperClass,
    max_seq_length=128,
    batch_size=8,
    shuffle=False
)

# ==============================
# Evaluate Model
# ==============================
prompt_model.eval()
all_preds = []
all_labels = []

with torch.no_grad():
    for batch in eval_loader:
        logits = prompt_model(batch)
        preds = torch.argmax(logits, dim=-1)
        all_preds.extend(preds.cpu().tolist())
        all_labels.extend(batch['label'].cpu().tolist())

# ==============================
# Print Classification Report
# ==============================
print("\n📊 XNLI Urdu Dev Classification Report:")
print(classification_report(all_labels, all_preds, target_names=classes, digits=4))

tokenizing: 2490it [00:00, 2614.27it/s]



📊 XNLI Urdu Dev Classification Report:
               precision    recall  f1-score   support

   entailment     0.3341    0.8651    0.4820       830
      neutral     0.3068    0.0651    0.1074       830
contradiction     0.3758    0.0747    0.1246       830

     accuracy                         0.3349      2490
    macro avg     0.3389    0.3349    0.2380      2490
 weighted avg     0.3389    0.3349    0.2380      2490



In [ ]:
roberta
📊 XNLI Urdu Dev Classification Report:
               precision    recall  f1-score   support

   entailment     0.3341    0.8651    0.4820       830
      neutral     0.3068    0.0651    0.1074       830
contradiction     0.3758    0.0747    0.1246       830

     accuracy                         0.3349      2490
    macro avg     0.3389    0.3349    0.2380      2490
 weighted avg     0.3389    0.3349    0.2380      2490

In [ ]:
mbert
📊 XNLI Urdu Dev Classification Report:
               precision    recall  f1-score   support

   entailment     0.3718    0.1048    0.1635       830
      neutral     0.3368    0.9084    0.4914       830
contradiction     0.2941    0.0060    0.0118       830

     accuracy                         0.3398      2490
    macro avg     0.3342    0.3398    0.2222      2490
 weighted avg     0.3342    0.3398    0.2222      2490

In [ ]:
# # ==============================
# # Load Evaluation Dataset
# # ==============================
# df = pd.read_csv(r"C:\Users\stdFurqan\Desktop\paft\SST-2\urdu_sentiment_test_labeled.csv")
# eval_dataset = [
#     InputExample(guid=i, text_a=row['text'], label=label_map[row['label']])
#     for i, row in df.iterrows()
# ]

# # ==============================
# # 0-Shot Evaluation with Each Template
# # ==============================
# prompt_model.eval()  # ensure model is in evaluation mode
# batch_size = 8    # eval batch size

# # Optional: store template order and results
# all_pass_patterns = {}

# for pass_idx, (prompt_name, current_template) in enumerate(templates, start=1):
#     print(f"\n🟦 0-Shot Evaluation - Template {prompt_name} ({pass_idx}/{len(templates)})")

#     # Create PromptDataLoader with current template
#     eval_loader = PromptDataLoader(
#         dataset=eval_dataset,
#         tokenizer=tokenizer,
#         template=current_template,
#         tokenizer_wrapper_class=WrapperClass,
#         max_seq_length=128,
#         batch_size=batch_size,
#         shuffle=False
#     )

#     pass_preds = []
#     pass_labels = []

#     # Run evaluation
#     with torch.no_grad():
#         for batch in eval_loader:
#             logits = prompt_model(batch)
#             preds = torch.argmax(logits, dim=-1)
#             pass_preds.extend(preds.cpu().tolist())
#             pass_labels.extend(batch['label'].cpu().tolist())

#     # Print report immediately after this template
#     print(f"\n📊 STS_B Urdu Dev Classification Report - Template {prompt_name}")
#     print(classification_report(pass_labels, pass_preds, target_names=classes, digits=4))

#     # Store template name (optional)
#     all_pass_patterns[f"pass_{pass_idx}"] = prompt_name

# # Optional: print template order at the end
# print("\n✅ Templates used per pass:", all_pass_patterns)
